<a href="https://colab.research.google.com/github/ThanuMahee12/ayush-vision/blob/yolo/Algorithums/detection/Yolo/YOLO_FLOWER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLO Flower

## Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
basicdata={'account':'sanashajeeva@gmail.com','user':"Yuka"}


In [ ]:
! locale-gen en_US.UTF-8
! export LC_ALL=en_US.UTF-8

Generating locales (this might take a while)...
  en_US.UTF-8... done
Generation complete.


## Intallations

In [ ]:
!pip install firebase-admin

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

## Imports

In [ ]:
import firebase_admin


In [ ]:
from firebase_admin import credentials,storage


In [ ]:
from firebase_admin import firestore

In [ ]:
from ultralytics import YOLO

In [ ]:
from datetime import date,datetime

In [ ]:
import os

In [ ]:
import numpy as np

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import random

In [ ]:
import yaml

In [ ]:
import csv

In [ ]:
import json

## Config

In [ ]:
epochs=10
imageSize=640
optimizer='Adam'
dropout=0.1

### Dirctory

In [ ]:
work_dirctory="/content/drive/MyDrive/AyushVision/yolo8/training"
work_dirctory_root="/content/drive/MyDrive/AyushVision/yolo8"
firebase_key="/content/drive/MyDrive/AyushVision/ServiceKey.json"

In [ ]:
today_work_dirctory=f"{work_dirctory}/{date.today().strftime('%d_%m_%Y')}"

In [ ]:
if not os.path.exists(work_dirctory):
  os.makedirs(work_dirctory,exist_ok=True)
if not os.path.exists(today_work_dirctory):
  os.makedirs(today_work_dirctory,exist_ok=True)

In [ ]:
data="/content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/data.yaml"

In [ ]:
project=f"{today_work_dirctory}/flowers/runs"
name=f'train{epochs}'

## Firebase

In [ ]:
cred = credentials.Certificate(firebase_key)
firebase_admin.initialize_app(cred,{
    'storageBucket': 'ayush-vision-asw4gh.appspot.com'
})

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [ ]:
db = firestore.client()

In [ ]:
bucket = storage.bucket()

In [ ]:
yolo_ref = db.collection("training").document('yolo')
doc = yolo_ref.get()

In [ ]:
if not doc.exists:
  doc_ref= db.collection("training").document('yolo')
  doc_ref.set({'algorithum':'yolo','version':'V8','mode':'detection','url':'ultralytics'})
else:
  doc_ref= yolo_ref

### Roboflow

## Model

In [ ]:
today_ref=doc_ref.collection("Flower").document(datetime.now().strftime(f"%Y%m%d%H"))

In [ ]:
basicdata['imageSize']=imageSize
basicdata['project']=project
basicdata['name']=name
basicdata['training_date']=datetime.now().strftime("%Y/%m/%d %H:%M:%S")
basicdata['filename']='YOLO_FLOWER'

In [ ]:
with open(data, 'r') as file:
    yaml_content = yaml.safe_load(file)

In [ ]:
basicdata['names']=yaml_content['names']

{'names': ['caster',
  'Indian_mallow',
  'Ivy_gourd',
  'Lime',
  'Long_pepper',
  'Pedaliummurex',
  'Red_Pea_Eggplant',
  'Ring_worm_plant',
  'ThornApple'],
 'nc': 9,
 'roboflow': {'license': 'CC BY 4.0',
  'project': 'ayushvision_rawfruit',
  'url': 'https://universe.roboflow.com/research-piohe/ayushvision_rawfruit/dataset/2',
  'version': 2,
  'workspace': 'research-piohe'},
 'test': '../test/images',
 'train': '/content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_rawfruit/train/images',
 'val': '/content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_rawfruit/valid/images'}

In [ ]:
today_ref.set(basicdata)

update_time {
  seconds: 1723272808
  nanos: 917362000
}

In [ ]:
yolo_model = YOLO('yolov8n.yaml').load('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 77.9MB/s]

Transferred 355/355 items from pretrained weights


#### Training

In [ ]:
results = yolo_model.train(data=data,name=name, epochs=epochs, imgsz=imageSize,project=project,optimizer=optimizer,plots=True,dropout=dropout)

Ultralytics YOLOv8.2.75 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_rawfruit/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/AyushVision/10_08_2024/fruits/runs, name=train10, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.1, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False

100%|██████████| 755k/755k [00:00<00:00, 14.5MB/s]


Overriding model.yaml nc=80 with nc=9

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_rawfruit/train/labels... 3773 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3773/3773 [14:45<00:00,  4.26it/s]


train: New cache created: /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_rawfruit/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

      10/10      2.27G      1.416      1.424      1.619         14        640: 100%|██████████| 236/236 [01:50<00:00,  2.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:05<00:00,  2.30it/s]

                   all        360        456      0.754      0.605      0.696      0.435



10 epochs completed in 0.336 hours.
Optimizer stripped from /content/drive/MyDrive/AyushVision/10_08_2024/fruits/runs/train10/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/AyushVision/10_08_2024/fruits/runs/train10/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/AyushVision/10_08_2024/fruits/runs/train10/weights/best.pt...
Ultralytics YOLOv8.2.75 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:11<00:00,  1.04it/s]


                   all        360        456      0.755      0.605      0.696      0.435
                caster         80        105      0.774      0.562      0.631      0.361
         Indian_mallow         40         48      0.926      0.524      0.794      0.456
             Ivy_gourd         40         45      0.808      0.842       0.86       0.68
                  Lime         40         48      0.504      0.646      0.519      0.254
           Long_pepper         40         61      0.781      0.557      0.736      0.461
         Pedaliummurex         41         63      0.591     0.0635      0.274      0.133
      Red_Pea_Eggplant         40         44      0.741       0.91      0.928      0.549
       Ring_worm_plant         40         42      0.912      0.737      0.828      0.587
Speed: 0.5ms preprocess, 3.7ms inference, 0.0ms loss, 7.6ms postprocess per image
Results saved to /content/drive/MyDrive/AyushVision/10_08_2024/fruits/runs/train10


In [ ]:
args_yaml=f'{project}/{name}/args.yaml'

In [ ]:
resultcsv=f'{project}/{name}/results.csv'

In [ ]:
with open(args_yaml, 'r') as file:
    yaml_content = yaml.safe_load(file)

In [ ]:
today_training_colref=today_ref.collection('Train')

In [ ]:
today_training_ref=today_training_colref.document('args')

In [ ]:
today_training_ref.set(yaml_content)

In [ ]:
todayresultref=today_training_colref.document('results')

In [ ]:
todayresultref.set({'results':data_dict})

In [ ]:
trainfiles={}
wights={}

In [ ]:
for file in os.listdir(f"{project}/{name}"):
  if os.path.isfile(f"{project}/{name}/{file}"):
    blob = bucket.blob(f"yolo/flower/train/{epochs}/{file}")
    blob.upload_from_filename(f"{project}/{name}/{file}")
    blob.make_public()
    trainfiles[os.path.splitext(file)[0]]=blob.public_url


In [ ]:
for file in os.listdir(f"{project}/{name}/weights"):
  if os.path.isfile(f"{project}/{name}/weights/{file}"):
    blob = bucket.blob(f"yolo/flower/train/{epochs}/weights/{file}")
    blob.upload_from_filename(f"{project}/{name}/weights/{file}")
    blob.make_public()
    wights[os.path.splitext(file)[0]]=blob.public_url

In [ ]:
today_training_files=today_training_colref.document('files')
today_training_files.set(trainfiles)

In [ ]:
today_weights_files=today_training_colref.document('weights')
today_weights_files.set(wights)

#### Validation

In [ ]:
model_train_path=f'{project}/{name}/weights/best.pt'

In [ ]:
model = YOLO(model_train_path)  # load a custom trained model

In [ ]:
metrics = model.val(save_json=True,plots=True,project=project,name=f'val{epochs}')  # no arguments needed, dataset and settings remembered

Ultralytics YOLOv8.2.75 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_rawfruit/valid/labels.cache... 360 images, 0 backgrounds, 0 corrupt: 100%|██████████| 360/360 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:14<00:00,  1.59it/s]


                   all        360        456      0.754      0.605      0.696      0.435
                caster         80        105      0.773      0.562      0.631       0.36
         Indian_mallow         40         48      0.926      0.524      0.794      0.456
             Ivy_gourd         40         45      0.808      0.842       0.86       0.68
                  Lime         40         48      0.504      0.646      0.519       0.26
           Long_pepper         40         61      0.781      0.557      0.736       0.46
         Pedaliummurex         41         63      0.589     0.0635       0.27      0.131
      Red_Pea_Eggplant         40         44      0.741       0.91      0.928      0.547
       Ring_worm_plant         40         42      0.912      0.738      0.828      0.588
Speed: 0.6ms preprocess, 8.1ms inference, 0.0ms loss, 5.4ms postprocess per image
Saving /content/drive/MyDrive/AyushVision/10_08_2024/fruits/runs/val10/predictions.json...
Results saved to /content/

In [ ]:
val_today_col=today_ref.collection('Val')

In [ ]:
predication_file=f'{project}/val{epochs}/predictions.json'

In [ ]:
with open(predication_file, 'r') as file:
    data_dict = json.load(file)

In [ ]:
val_today_ref_files=val_today_col.document('files')

In [ ]:
valfiles={}

In [ ]:
for file in os.listdir(f"{project}/val{epochs}"):
  if os.path.isfile(f"{project}/val{epochs}/{file}"):
    blob = bucket.blob(f"yolo/flowers/val/{epochs}/{file}")
    blob.upload_from_filename(f"{project}/val{epochs}/{file}")
    blob.make_public()
    valfiles[os.path.splitext(file)[0]]=blob.public_url

In [ ]:
val_today_ref_files.set(valfiles)

In [ ]:
val_today_ref_metrics=val_today_col.document('metricsbox')

In [ ]:
val_today_ref_metrics.set({'map50':metrics.box.map50,'map75':metrics.box.map75})

In [ ]:
val_today_ref_metrics.update({
    'fitness':metrics.fitness
})

In [ ]:
maps=np.array(metrics.maps).tolist()
boxmaps=np.array(metrics.box.maps).tolist()
p=np.array(metrics.box.p).tolist()

In [ ]:
val_today_ref_metrics.update({
    'metricsmaps':maps,
    'maps':boxmaps,
    'p':p
})

In [ ]:
metrics.box.map50

0.8589041190345956

In [ ]:
metrics.box.map75

0.639577746476327

In [ ]:
metrics.box.maps

array([     0.5406,     0.57343,     0.74344,     0.49199,     0.57389,     0.24103,     0.66876,      0.7052])

#### Predict

In [ ]:

results=model.predict(source=f'{work_dirctory_root}/dataset/ayushvision_root/test/images',save=True)

In [ ]:
predicts=[]

In [ ]:
for file in os.listdir(results[0].save_dir):
  if os.path.isfile(f"{results[0].save_dir}/{file}"):
    blob = bucket.blob(f"yolo/root/predict/{epochs}/{file}")
    blob.upload_from_filename(f"{results[0].save_dir}/{file}")
    blob.make_public()
    predicts.append(blob.public_url)

160

In [ ]:
today_ref.collection('Predict').add({
    'files':predicts
})

In [ ]:
predicts=[]

In [ ]:
for file in os.listdir(f"{today_work_dirctory}/predicts{epochs}"):
  if os.path.isfile(f"{today_work_dirctory}/predicts{epochs}/{file}"):
    blob = bucket.blob(f"yolo/flower/predict/{epochs}/{file}")
    blob.upload_from_filename(f"{today_work_dirctory}/predicts{epochs}/{file}")
    blob.make_public()
    predicts.append(blob.public_url)

In [ ]:
today_ref.collection('Predict').add({
    'files':predicts
})

### Export Model

In [ ]:
format='onnx'
model.export(format=format,keras=True,dynamic=True,simplify=True,)

Ultralytics YOLOv8.2.74 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (Intel Xeon 2.20GHz)

PyTorch: starting from '/content/drive/MyDrive/Research/07_08_2024/fruits/runs/train30/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 12, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim>=0.1.31', 'onnxruntime'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 191.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 186.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 264.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 165.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 238.0 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 11.5s, installed 3 packages: ['onnx>=1.12.0', 'onnxslim>=0.1.31', 'onnxruntime']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: 

'/content/drive/MyDrive/Research/07_08_2024/fruits/runs/train30/weights/best.onnx'

In [ ]:
blob = bucket.blob(f"yolo/root/model/{epochs}/best.onnx")
blob.upload_from_filename(f"{today_work_dirctory}/roots/runs/{name}/weights/best.onnx")
blob.make_public()

In [ ]:
today_ref.collection('Model').add({
    'format':format,
    "keras":True,
    "dynamic":True,
    "simplify":True,
    "model":blob.public_url
})